In [ ]:
%pip install pandas seaborn matplotlib

In [ ]:
import pandas as pd

JOIN_KEYS = ["año", "capitulo", "nombre_capitulo"]

presupuesto_df = (
    pd.read_csv("../datasets/pagos_salud_chile_2018-2024.csv", delimiter=",")
    .rename(columns={"devengo": "total"})
    .groupby(["año", "capitulo", "nombre_capitulo", "subtitulo", "nombre_subtitulo"])[
        ["total"]
    ]
    .agg("sum")
    .reset_index()
)

subtitulo_columns = (
    presupuesto_df[["subtitulo", "nombre_subtitulo"]]
    .sort_values(by="subtitulo")
    .drop_duplicates()["nombre_subtitulo"]
    .to_list()
)

presupuesto_df = (
    presupuesto_df.pivot_table(
        index=["año", "capitulo", "nombre_capitulo"],
        columns=["nombre_subtitulo"],
        values=["total"],
    )
    .fillna(0)
    .reset_index()
)

presupuesto_df = pd.concat(
    [
        presupuesto_df[["año", "capitulo", "nombre_capitulo"]],
        presupuesto_df["total"][subtitulo_columns],
    ],
    axis=1,
)
presupuesto_df.columns = [
    "año",
    "capitulo",
    "nombre_capitulo",
] + presupuesto_df.columns[3:].to_list()
presupuesto_df.columns = presupuesto_df.columns.str.lower().str.replace(" ", "_")
presupuesto_df

In [ ]:
intento_ideacion_df = pd.read_csv("../datasets/ideacion_intento.csv", delimiter=",")
urgencia_autolesiones_df = pd.read_csv(
    "../datasets/urgencia_lesiones_autoinflingidas.csv", delimiter=","
)

joined_df = pd.merge(presupuesto_df, intento_ideacion_df, on=JOIN_KEYS, how="outer")
joined_df = pd.merge(joined_df, urgencia_autolesiones_df, on=JOIN_KEYS, how="outer")
joined_df

In [ ]:
joined_df = joined_df.query("año >= 2018 and año <= 2023")
joined_df